This jupyter notebook creates the dataset from the Syllabi File. Please visit the README file to learn the naming convention for the syllabi.

Install necassary libraries

In [ ]:
%pip install PyPDF2
%pip install textract
%pip install pdf2docx
%pip install pyChatGPT
%pip install openai;

Import the libraries needed

In [ ]:
from PyPDF2 import PdfReader
import csv
from pdf2docx import Converter
import textract
import re
import string 
import docx2txt
from pyChatGPT import ChatGPT
import time
import os
import openai

Set path to the syllabi folder and print current files in Syllabi Directory.

In [ ]:
#Locate and Get file names out of directory
Path = "Syllibi/"
DirList = os.listdir(Path)
#DirList.remove('.DS_Store')
print(DirList)


Remove all special characters function used to clean up the passages on they are created.

In [ ]:
def removeSpecialCharacter(s):
    t = ""
    la="abcdefghijklmnopqrstuvwxyz"
    ua="ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    nu="0123456789"
    space = ' '
    endline = '\n\r\t'
    punc=string.punctuation
    for i in s:
        if(i in la or i in ua or i in punc or i in nu or i in space or i in endline):
            t+=i
    return t

ExtractTitle function is used to parse through the naming convention and assign the information to variables. These features are added to the dataset.


In [ ]:
def extracttitle(Filetitle):
    Extractedlist = Filetitle.split(",")
    print(Extractedlist)
    if (len(Extractedlist) != 5):
        return 0
    classtype = Extractedlist[0]
    classnumber = Extractedlist[1]
    classtitle = Extractedlist[2]
    year = Extractedlist[3]
    professor = Extractedlist[4]
    professor = professor[:-5]
    print(classtype, classnumber, classtitle, year, professor)
    return classtype, classnumber, classtitle, professor, year


Experimenting with third party Chat GPT request

In [ ]:

#session_token = ""
#api = ChatGPT(auth_type='google', email='', password='')
#api = ChatGPT(session_token)

Assign openai api key for text-divinci-003

In [ ]:
openai.api_key = ""
openai.Model.retrieve("text-davinci-003")

Convert all the files to docx

In [ ]:
for x in range(0,len(DirList)):
    if ".pdf" in DirList[x]:
        tempe = DirList[x]
        path_input = Path + DirList[x]
        path_output = Path + tempe[:-4] + '.docx'
        cv = Converter(path_input)
        cv.convert(path_output, start=0, end=None)
        cv.close()
        os.remove(path_input)


Creating the Dataset

In [ ]:
#redefine File List
DirList = os.listdir(Path)
counter = 0
file = open('SyllabiDataset.csv', 'w', newline ='')

for x in range(0,len(DirList)):
    counter += 1

    #Make sure the file is in .docx form
    if ".docx" in DirList[x]:

        #extract dataset information from syllabis title
        department, number, name, professor, year = extracttitle(DirList[x])

        # Create the Path to the Doc File
        temp = ''
        tempdoc = Path + DirList[x]
        
        # extract the text then convert byte object to string
        text = docx2txt.process(tempdoc)
        text = removeSpecialCharacter(text)
        
        #Split by tab and enter
        Splittext = re.split('\n|\t',text)
        
        #remove empty strings and then remove unwanted spaces
        while("" in Splittext):
            Splittext.remove('')
        for p in range(0,len(Splittext)):
            Splittext[p] = re.sub("\s\s+" , " ", Splittext[p])
       
        #Use 1000 character threshold to combine strings.
        NewList = []
        NewList.append(Splittext[0])
        for y in range(1, len(Splittext)):
            if (len(NewList[-1]) < 1000):
                NewList[-1] = (NewList[-1] + ' ' + Splittext[y])
            else:
                NewList.append(Splittext[y])
        
        # writing the data into the csv file
        file = open('SyllabiDataset.csv', 'a', newline ='') 
        
        #open file
        with file:

            #Go through dataset and create 3 question,answer, and keyword pairs using openai text-divinci-003
            for x in range(1,len(NewList)):

                #Send API request
                prom = "Create 3 questions that each have a short answer and 3 to 5 keywords in the format 'Question: >> Answer: >> keywords:'. Dont apply any formatting in the response only follow the exact format stated do not split Question: >> Answer: >> keywords:. base the question on this text: " + NewList[x]
                resp = openai.Completion.create(
                    model="text-davinci-003",
                    prompt=prom,
                    max_tokens=3000
                        )
                
                #extract response from API call
                QA = resp['choices'][0]['text']
                #split by new line to get the pairs
                QAnew = re.split('\n',QA)
                while("" in QAnew):
                    QAnew.remove('')
                while(' ' in QAnew):
                    QAnew.remove(' ')

                #Check to make sure the paragraphs have all three pairs of Question and Answer
                for elements in QAnew:
                    if "QA1" in elements:
                        pass
                    elif "QA2" in elements:
                        pass
                    elif "QA3" in elements:
                        pass
                    elif "Q" in elements:
                        pass
                    elif "Q1." in elements:
                        pass
                    elif "Q2" in elements:
                        pass
                    elif "Q3" in elements:
                        pass
                    elif "Question." in elements:
                        pass
                    elif "Question:" in elements:
                        pass
                    elif "Question 1:" in elements:
                        pass
                    elif "Question 2:" in elements:
                        pass
                    elif "Question 3:" in elements:
                        pass
                    else:
                        QAnew.remove(elements)
                print(QAnew)
                if(len(QAnew) != 3):
                    continue
                QA1 = re.split('>>', QAnew[0])
                if(len(QA1) != 3):
                    continue 
                QA2 = re.split('>>', QAnew[1])
                if(len(QA2) != 3):
                    continue 
                QA3 = re.split('>>', QAnew[2])
                if(len(QA3) != 3):
                    continue
                    
                #API request limit per minute so sleep for 2 seconds to prevent API request failure
                time.sleep(2)
                    # QA4 = re.split('>', QAnew[3])
                    # print(len(QA4))
                    # QA5 = re.split('>', QA[4])    
                    #api.refresh_auth()
                    #api.clear_conversations()

                #Write the question/answer/keywords to corresponding passage entries
                improvemen = "This class is "+department+ ' '+ number+ ', '+ name+ ', taught by '+professor+ ' in '+ year+'. '  
                fieldnames = ['text', 'class type', 'class number','class name', 'class professor', 'class year', 'Q1', 'A1', 'K1', 'Q2', 'A2', 'K2', 'Q3', 'A3', 'K3']#, 'Q4', 'A4', 'K4']#, 'Q5', 'A5', 'K5']
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writerow({'text':improvemen + NewList[x], 'class type': department, 'class number': number,'class name': name, 'class year': year, 'class professor': professor , 'Q1': QA1[0], 'A1': QA1[1], 'K1': QA1[2], 'Q2': QA2[0], 'A2': QA2[1], 'K2': QA2[2], 'Q3': QA3[0], 'A3': QA3[1], 'K3': QA3[2]})#, 'Q4': QA4[0], 'A4': QA4[1], 'K4': QA4[2]})#, 'Q5': QA5[0], 'A5': QA5[1], 'K5': QA5[2]})

        
